## Google Scholar Crawler
*DISCLAIMER: after retrieving 490 URLs, Google BLOCKED the IP address

*TODO: add selenium and add a manual step for opening a url and showing that we are NOT a robot

In [122]:
keyword_raw = "ebpf ddos xdp"


In [127]:
import cfscrape
from lxml import etree
import pandas as pd
import urllib
import re
from random import choice, randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from __future__ import print_function
from IPython.display import display
from ipywidgets import interact,Checkbox, Button


#15 most common user-agents from: https://techblog.willshouse.com/2012/01/03/most-common-user-agents/
user_agents = [
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1 Safari/605.1.15',
'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36',
'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0',
'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0',
'Mozilla/5.0 (Windows NT 6.1; rv:60.0) Gecko/20100101 Firefox/60.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
]

In [128]:
#download drivers: https://www.seleniumhq.org/download/


profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", choice(user_agents))

firefoxDriver = webdriver.Firefox(profile)
#chromeDriver = webdriver.Chrome()

print('Current userAgent:', firefoxDriver.execute_script("return navigator.userAgent"))

Current userAgent: Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0


In [129]:
#function to change User-Agent on Firefox driver
def change_user_agent(driver):
    driver.get("about:config")
    useragent = choice(user_agents)
    jscript = """navigator.__defineGetter__('userAgent', function () {return "%s"});""" %useragent
    driver.execute_script(jscript)

    print('Current userAgent:', driver.execute_script("return navigator.userAgent"))

In [130]:
page = 1 
df = pd.DataFrame()
    
while True: 
    start_page = (page - 1) * 10
    #################################################################################
    keyword = urllib.parse.quote_plus(keyword_raw)
    url="https://scholar.google.nl/scholar?start="+str(start_page)+"&q="+keyword+"&hl=nl&as_sdt=0,5"
    print(page, url)
    #################################################################################    
    
    firefoxDriver.get(url)
    scraped_html = firefoxDriver.page_source
    
    # print(scraped_html.decode("utf-8") )
    html = etree.HTML(scraped_html)
    #################################################################################    
    elements = html.xpath("//div[@class='gs_ri']")
    #################################################################################
    if len(elements) == 0:
        print("The code stopped for some reason!! Possibly Google blocked us!")
        print("\""+(''.join(html.xpath("//div[@id='infoDiv']/text()")))+"\"")
        inp = input("(Empty -> Continue),  ('new' -> new User-Agent and Continue), (Any other character -> Break")
        if not inp:
            continue
        elif inp == 'new':
            change_user_agent(firefoxDriver)
            continue
        else:
            break
    #################################################################################
    for element in elements:
        #################################################################################
        if len(element.xpath("h3[@class='gs_rt']/a/text()")) > 1:
            title = (keyword_raw+" ").join(element.xpath("h3[@class='gs_rt']/a/text()"))
        elif len(element.xpath("h3[@class='gs_rt']/a/text()")) == 1:
            title = element.xpath("h3[@class='gs_rt']/a/text()")[0]
        else:
            title = ""
        #################################################################################    
        link  = ''.join(element.xpath("h3[@class='gs_rt']/a/@href"))
        #################################################################################    
        abstract = ''.join(element.xpath("div[@class='gs_rs']/text()")).replace('\n','')
        #################################################################################
        reference = ''.join(element.xpath("div[@class='gs_a']/text()"))
        #################################################################################
        try:
            year = re.search(r"\d{4}", reference)[0]
        except:
            year = ""
        #################################################################################   
        df = df.append({'year':year,'title':title,'abstract':abstract,'reference':reference,'link':link},ignore_index=True)
    #################################################################################   
    print("sleeping...")
    sleep(randint(6,18))
    #################################################################################   
    page = page + 1

firefoxDriver.quit()

1 https://scholar.google.nl/scholar?start=0&q=ebpf+ddos+xdp&hl=nl&as_sdt=0,5
sleeping...
2 https://scholar.google.nl/scholar?start=10&q=ebpf+ddos+xdp&hl=nl&as_sdt=0,5
sleeping...
3 https://scholar.google.nl/scholar?start=20&q=ebpf+ddos+xdp&hl=nl&as_sdt=0,5
sleeping...
4 https://scholar.google.nl/scholar?start=30&q=ebpf+ddos+xdp&hl=nl&as_sdt=0,5
The code stopped for some reason!! Possibly Google blocked us!
""
(Empty -> Continue),  ('new' -> new User-Agent and Continue), (Any other character -> Breaknew
Current userAgent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36
4 https://scholar.google.nl/scholar?start=30&q=ebpf+ddos+xdp&hl=nl&as_sdt=0,5
The code stopped for some reason!! Possibly Google blocked us!
""
(Empty -> Continue),  ('new' -> new User-Agent and Continue), (Any other character -> Breakasd


## Exporting the output to a csv file

In [132]:
filename = urllib.parse.quote_plus(keyword_raw)+'_papers_crawler_google_scholar.csv'

df.to_csv(filename)

## Further analysis

In [133]:
pd.options.display.max_colwidth = 100
from urllib.parse import urlparse

In [138]:
#Reading again the file in case it is a offline analysis
df = pd.read_csv(filename,index_col=0).dropna()

In [139]:
len(df)

17

In [140]:
print("TOTAL ENTRIES:",len(df))

TOTAL ENTRIES: 17


In [141]:
source_distribution = df['link'].dropna().apply(lambda x: urlparse(x)[1].replace("www.","")).value_counts()
print("Distinct Source of Information:",len(source_distribution))
print()
print("Frequency of Source of Information:")
print(source_distribution.head(30))

Distinct Source of Information: 11

Frequency of Source of Information:
dl.acm.org                   3
diva-portal.org              2
linuxplumbersconf.org        2
ieeexplore.ieee.org          2
arxiv.org                    2
minlanyu.seas.harvard.edu    1
researchgate.net             1
duo.uio.no                   1
brendangregg.com             1
netdevconf.org               1
sciencedirect.com            1
Name: link, dtype: int64


The most important source of information are: 
- ieeexplore.ieee.org
- dl.acm.org
- link.springer.com
- pdfs.semanticscholar.org
- sciencedirect.com
- researchgate.net

Of course the RFCs of IETF are also very important:
- rfc-editor.org 
- tools.ietf.org
- datatracker.ietf.org

## Exporting information to csv files based on the source of information

In [40]:
df[df['link'].str.contains("ieeexplore.ieee.org")].to_csv("flowspec_ieee.csv")

In [41]:
df[df['link'].str.contains("link.springer.com")].to_csv("flowspec_springer.csv")

In [42]:
df[df['link'].str.contains("pdfs.semanticscholar.org")].to_csv("flowspec_semanticscholar.csv")

In [45]:
df[df['link'].str.contains("dl.acm.org")].to_csv("flowspec_acm.csv")

## Lessons learned:
- remove papers that have in the title or description/abstract the list of words: [rsvp, qos, quality of service, mpls, intserv, diffserv, multicast]